# Data Collecting

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pylab as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv('../data/stroke.csv')

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [3]:
label = 'stroke'
df[label].value_counts()/df.shape[0]

0    0.981959
1    0.018041
Name: stroke, dtype: float64

The balance of data is low.

# Data Preprocessing

'id' identifies unique individual. It is not a contributing factor to stroke. So I will drop the 'id' column.

In [4]:
df.drop(columns=['id'], inplace=True)

In [ ]:
df.isnull().sum(axis=0)

In [ ]:
df.isnull().sum(axis=0)/df.shape[0]*100

The dataset contains missing values in 'bmi' and 'smoking_status'.

People who do not fill in 'smoking_status' information may have similar reasons why they don't. So I will replace the missing values in 'smoking_status' with 'missing' as a separate category.

In [5]:
label = 'stroke'
onehot_ftrs = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
num_ftrs = ['age', 'avg_glucose_level']
bin_ftrs = ['hypertension', 'heart_disease']

In [6]:
def preprocess(X_train, X_CV, X_test, random_state):
    onehot_enc = OneHotEncoder(sparse=False, categories='auto', handle_unknown='ignore')
    cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')
    onehot_encoded = onehot_enc.fit_transform(cat_imputer.fit_transform(X_train[onehot_ftrs]))
    X_train_oh = pd.DataFrame(onehot_encoded, columns=onehot_enc.get_feature_names())
    X_CV_oh = onehot_enc.transform(cat_imputer.transform(X_CV[onehot_ftrs]))
    X_CV_oh = pd.DataFrame(X_CV_oh, columns=onehot_enc.get_feature_names())
    X_test_oh = onehot_enc.transform(cat_imputer.transform(X_test[onehot_ftrs]))
    X_test_oh = pd.DataFrame(X_test_oh, columns=onehot_enc.get_feature_names())
                
    scaler = MinMaxScaler()
    iter_imputer = IterativeImputer(estimator = RandomForestRegressor(n_estimators=100), 
                                    random_state = random_state)
    mm_scaled = scaler.fit_transform(iter_imputer.fit_transform(X_train[num_ftrs]))
    X_train_num = pd.DataFrame(mm_scaled, columns=num_ftrs)
    X_CV_num = scaler.transform(iter_imputer.transform(X_CV[num_ftrs]))
    X_CV_num = pd.DataFrame(X_CV_num, columns=num_ftrs)
    X_test_num = scaler.transform(iter_imputer.transform(X_test[num_ftrs]))
    X_test_num = pd.DataFrame(X_test_num, columns=num_ftrs)
        
    X_train_bin = X_train[bin_ftrs]
    X_CV_bin = X_CV[bin_ftrs]
    X_test_bin = X_test[bin_ftrs]
    X_train_bin.reset_index(drop=True, inplace=True)
    X_CV_bin.reset_index(drop=True, inplace=True)
    X_test_bin.reset_index(drop=True, inplace=True)
        
    X_train_ = pd.concat([X_train_bin, X_train_oh, X_train_num], axis=1)
    X_CV_ = pd.concat([X_CV_bin, X_CV_oh, X_CV_num], axis=1)
    X_test_ = pd.concat([X_test_bin, X_test_oh, X_test_num], axis=1) 
    
    return X_train_, X_CV_, X_test_

# Models

### metric baselines

In [7]:
# precision
base_precision = len(df[df[label]==1])/df.shape[0]
base_recall = 1
base_f1 = 2*base_precision*base_recall/(base_precision+base_recall)
print('precision base line is {}.'.format(base_precision))
print('recall base line is {}.'.format(base_recall))
print('f1 score base line is {}.'.format(base_f1))

precision base line is 0.01804147465437788.
recall base line is 1.
f1 score base line is 0.03544349636738112.


### logistic regression

In [8]:
y = df[label]
X = df.drop(columns=[label])

In [11]:
def ML_pipeline_kfold_logistic(X, y, random_state, n_folds):
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, stratify=y,
                                                        random_state=random_state)
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    
    f1_scores = []
    best_alphas_f1 = []

    for train_index, CV_index in kf.split(X_other, y_other):
        X_train, X_CV = X_other.iloc[train_index.tolist()], X_other.iloc[CV_index.tolist()]
        y_train, y_CV = y_other.iloc[train_index.tolist()], y_other.iloc[CV_index.tolist()]

        X_train_, X_CV_, X_test_ = preprocess(X_train, X_CV, X_test, random_state)
        
        alpha = np.logspace(-5, 2, num=20)
        clfs = []
        thresholds = []
        CV_score_f1 = []
        
        for a in alpha:
            clf = LogisticRegression(penalty='l1', C=1/a, max_iter=10000, solver='saga',
                                    multi_class='auto')
            clf.fit(X_train_, y_train)
            
            y_train_prob = clf.predict_proba(X_train_)
            thres = min([prob2 for [prob1, prob2] in y_train_prob])
            y_CV_prob = clf.predict_proba(X_CV_)
            y_CV_prob = [prob2 for [prob1, prob2] in y_CV_prob]
            y_CV_pred = [1 if prob>thres else 0 for prob in y_CV_prob]
            
            CV_score_f1.append(f1_score(y_CV, y_CV_pred))
            thresholds.append(thres)
            clfs.append(clf)
        
        best_alpha_f1 = alpha[np.argmax(CV_score_f1)]
        best_alphas_f1.append(best_alpha_f1)
        clf = clfs[np.argmax(CV_score_f1)]
        thres = thresholds[np.argmax(CV_score_f1)]
        y_test_prob = clf.predict_proba(X_test_)
        y_test_prob = [prob2 for [prob1, prob2] in y_test_prob]
        y_test_pred = [1 if prob>thres else 0 for prob in y_test_prob]
        f1_scores.append(f1_score(y_test,y_test_pred))

    best_a_f1 = best_alphas_f1[np.argmax(f1_scores)]
    return best_a_f1, f1_scores

f1_scores_l = []

for i in range(5):
    random_state = 42*i
    best_a_f1, f1_scores = ML_pipeline_kfold_logistic(X, y, random_state, 5)

    f1_scores_l.append(f1_scores)
    print('random state = {}:'.format(random_state))
    print('Best alpha for f1 score is {}'.format(best_a_f1))

mean_f1_l = np.mean(f1_scores_l)
std_f1_l = np.std(f1_scores_l)
print('test recall score:', np.around(mean_f1_l,3), '+/-', np.around(std_f1_l,3))

random state = 0:
Best alpha for f1 score is 42.81332398719387
random state = 42:
Best alpha for f1 score is 42.81332398719387
random state = 84:
Best alpha for f1 score is 42.81332398719387
random state = 126:
Best alpha for f1 score is 42.81332398719387
random state = 168:
Best alpha for f1 score is 42.81332398719387
test recall score: 0.036 +/- 0.0


In [12]:
def ML_pipeline_kfold_random_forest(X, y, random_state, n_folds):
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, stratify=y,
                                                        random_state = random_state)
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    f1_scores = []
    best_paras_f1 = []

    for train_index, CV_index in kf.split(X_other, y_other):
        X_train, X_CV = X_other.iloc[train_index.tolist()], X_other.iloc[CV_index.tolist()]
        y_train, y_CV = y_other.iloc[train_index.tolist()], y_other.iloc[CV_index.tolist()]
        
        X_train_, X_CV_, X_test_ = preprocess(X_train, X_CV, X_test, random_state)
        
        paras = [(d, s) for d in range(1, 10) for s in range(3, 15)]
        clfs = []
        thresholds = []
        CV_score_f1 = []

        for d, s in paras:
            clf = RandomForestClassifier(n_estimators=100, random_state=random_state, 
                                 max_depth=d, min_samples_split=s)
            clf.fit(X_train_, y_train)
            y_train_prob = clf.predict_proba(X_train_)
            thres = min([prob2 for [prob1, prob2] in y_train_prob])
            y_CV_prob = clf.predict_proba(X_CV_)
            y_CV_prob = [prob2 for [prob1, prob2] in y_CV_prob]
            y_CV_pred = [1 if prob>thres else 0 for prob in y_CV_prob]

            CV_score_f1.append(f1_score(y_CV, y_CV_pred))
            thresholds.append(thres)
            clfs.append(clf)
        
        best_d_f1 = paras[np.argmax(CV_score_f1)][0]
        best_s_f1 = paras[np.argmax(CV_score_f1)][1]
        best_paras_f1.append((best_d_f1, best_s_f1))
        clf = clfs[np.argmax(CV_score_f1)]
        thres = thresholds[np.argmax(CV_score_f1)]
        y_test_prob = clf.predict_proba(X_test_)
        y_test_prob = [prob2 for [prob1, prob2] in y_test_prob]
        y_test_pred = [1 if prob>thres else 0 for prob in y_test_prob]
        f1_scores.append(f1_score(y_test,y_test_pred))
  
    best_para_f1 = best_paras_f1[np.argmax(f1_scores)]
    return best_para_f1, f1_scores  

f1_scores_rf = []
for i in range(8):
    random_state = 42*i
    best_para_f1, f1_scores = ML_pipeline_kfold_random_forest(X, y, random_state, 5)
    f1_scores_rf.append(f1_scores)
    print('random state = {}:.'.format(random_state))
    print('Best max_depth is {} and best min_samples_split is {}.'.format(best_para_f1[0], best_para_f1[1]))
    
mean_f1_rf = np.mean(f1_scores_rf)
std_f1_rf = np.std(f1_scores_rf)
print('test f1 score:', np.around(mean_f1_rf,3), '+/-', np.around(std_f1_rf,3))

NameError: name 'best_paras' is not defined

# EDA

In [ ]:
df = pd.read_csv('../data/stroke.csv')
df['smoking'].fillna(value='missing', inplace=True)

df['stroke'].replace(to_replace=0, value='not stroke', inplace=True)
df['stroke'].replace(to_replace=1, value='stroke', inplace=True)

df['hypertension'].replace(to_replace=0, value='No', inplace=True)
df['hypertension'].replace(to_replace=1, value='Yes', inplace=True)

df['heart_disease'].replace(to_replace=0, value='No', inplace=True)
df['heart_disease'].replace(to_replace=1, value='Yes', inplace=True)

### Numerical Feature -- age

In [ ]:
categories = df['stroke'].unique()
bin_range = (df['age'].min(), df['age'].max())

for c in categories:
    plt.hist(df[df['stroke']==c]['age'], alpha=0.5, label=c, range=bin_range, bins=20, density=True)
plt.legend()
plt.ylabel('pdf')
plt.xlabel('age')
plt.tight_layout()
plt.savefig('../figures/age_hist.png', dpi=300)
plt.show()

In [ ]:
dataset = [df[df['stroke']=='not stroke']['age'].values,
           df[df['stroke']=='stroke']['age'].values]

plt.violinplot(dataset = dataset)
plt.xticks([1,2], ['not stroke','stroke'])
plt.ylabel('age')
plt.savefig('../figures/age_violin.png', dpi=300)
plt.show()

### Numerical Feature -- avg_glucose_level

In [ ]:
categories = df['stroke'].unique()
bin_range = (df['avg_glucose_level'].min(), df['avg_glucose_level'].max())

for c in categories:
    plt.hist(df[df['stroke']==c]['avg_glucose_level'], alpha=0.5, label=c, range=bin_range, bins=20, density=True)
plt.legend()
plt.ylabel('pdf')
plt.xlabel('avg_glucose_level')
plt.axvline(x=70, color='r')
plt.axvline(x=126, color='r')
plt.tight_layout()
plt.savefig('../figures/avg_glucose_level_hist.png', dpi=300)
plt.show()

In [ ]:
dataset = [df[df['stroke']=='not stroke']['avg_glucose_level'].values,
           df[df['stroke']=='stroke']['avg_glucose_level'].values]

plt.violinplot(dataset = dataset)
plt.xticks([1,2], ['not stroke', 'stroke'])
plt.ylabel('avg_glucose_level')
plt.savefig('../figures/avg_glucose_level_violin.png', dpi=300)
plt.show()

### Numerical Feature -- bmi

In [ ]:
categories = df['stroke'].unique()
bin_range = (df['bmi'].min(), df['bmi'].max())

for c in categories:
    plt.hist(df[df['stroke']==c]['bmi'], alpha=0.5,
             label=c, range=bin_range, bins=20, density=True)

plt.legend()
plt.ylabel('pdf')
plt.xlabel('bmi')
plt.axvline(x=18.5, color='r')
plt.axvline(x=25, color='r')
plt.tight_layout()
plt.savefig('../figures/bmi_hist.png', dpi=300)
plt.show()

### Categorical Feature -- hypertension

In [ ]:
count_matrix = df.groupby(['hypertension', 'stroke']).size().unstack()
count_matrix_norm = count_matrix.div(count_matrix.sum(axis=1), axis=0)
count_matrix_norm.plot(kind='bar', stacked=True, color=('steelblue', 'powderblue'))
plt.ylabel('fraction of people in group')
plt.ylim(0.93,1)
plt.savefig('../figures/hypertension_bar.png')
plt.show()

In [ ]:
sns.boxplot(x = df['hypertension'], y = df['age'], hue = df['stroke'], palette = 'Blues')
plt.savefig('../figures/hyper_age_stroke')

### Categorical Feature -- heart_disease

In [ ]:
count_matrix = df.groupby(['heart_disease', 'stroke']).size().unstack()
count_matrix_norm = count_matrix.div(count_matrix.sum(axis=1), axis=0)
count_matrix_norm.plot(kind='bar', stacked=True, color=('steelblue', 'powderblue'))
plt.ylabel('fraction of people in group')
plt.ylim(0.9,1)
plt.savefig('../figures/heart_disease_bar.png')
plt.show()

In [ ]:
sns.boxplot(x = df['heart_disease'], y = df['age'], hue = df['stroke'], palette = 'Blues')
plt.savefig('../figures/heart_age_stroke')

### Categorical Feature -- gender

In [ ]:
count_matrix = df.groupby(['gender', 'stroke']).size().unstack()
count_matrix_norm = count_matrix.div(count_matrix.sum(axis=1), axis=0)
count_matrix_norm.plot(kind='bar', stacked=True, color=('steelblue', 'powderblue'))
plt.ylabel('fraction of people in group')
plt.ylim(0.97, 1)
plt.savefig('../figures/gender_bar.png')
plt.show()

### Categorical Feature -- work_type

In [ ]:
count_matrix = df.groupby(['work_type', 'stroke']).size().unstack()
count_matrix_norm = count_matrix.div(count_matrix.sum(axis=1), axis=0)
count_matrix_norm.plot(kind='bar', stacked=True, color=('steelblue', 'powderblue'))
plt.ylabel('fraction of people in group')
plt.ylim(0.95, 1)
plt.savefig('../figures/work_type_bar.png')
plt.show()

### Categorical Feature -- smoking_status

In [ ]:
count_matrix = df.groupby(['smoking_status', 'stroke']).size().unstack()
count_matrix_norm = count_matrix.div(count_matrix.sum(axis=1), axis=0)
count_matrix_norm.plot(kind='bar', stacked=True, color=('steelblue', 'powderblue'))
plt.ylabel('fraction of people in group')
plt.ylim(0.96, 1)
plt.savefig('../figures/smoking_status_bar.png')
plt.show()

In [ ]:
sns.boxplot(x = df['smoking_status'], y = df['age'], hue = df['stroke'], palette = 'Blues')
plt.savefig('../figures/smoking_age_stroke')

### Categorical Feature -- Residence_type

In [ ]:
count_matrix = df.groupby(['Residence_type', 'stroke']).size().unstack()
count_matrix_norm = count_matrix.div(count_matrix.sum(axis=1), axis=0)
count_matrix_norm.plot(kind='bar', stacked=True, color=('steelblue', 'powderblue'))
plt.ylabel('fraction of people in group')
plt.ylim(0.96, 1)
plt.savefig('../figures/Residence_type_bar.png')
plt.show()

### Categorical Feature -- ever_married

In [ ]:
count_matrix = df.groupby(['ever_married', 'stroke']).size().unstack()
count_matrix_norm = count_matrix.div(count_matrix.sum(axis=1), axis=0)
count_matrix_norm.plot(kind='bar', stacked=True, color=('steelblue', 'powderblue'))
plt.ylabel('fraction of people in group')
plt.ylim(0.97, 1)
plt.savefig('../figures/ever_married_bar.png')
plt.show()

In [ ]:
sns.boxplot(x = df['ever_married'], y = df['age'], hue = df['stroke'], palette = 'Blues')
plt.savefig('../figures/ever_age_stroke')